## Data Profiling

In [12]:
import pandas as pd
from sqlalchemy import create_engine

In [13]:
db = {

"user" : "postgres",
"password":"admin123",
"port":5433,
"db":"mydb",
"host":'localhost',

}

engine = create_engine(f"postgresql+psycopg2://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['db']}")


#### 1. List all tables

In [ ]:
def list_tables (engine):
    query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public';
    """
    table_list = pd.read_sql(query, engine)
    print(table_list)
    return table_list

table_name = list_tables(engine)

           table_name
0            products
1  inventory_tracking
2              orders
3       order_details
4           customers
5           employees


,table_name
0,products
1,inventory_tracking
2,orders
3,order_details
4,customers
5,employees


#### 2. Extract Data

In [ ]:
def extract_data(engine,table_name):
    list_data = {}
    for row in table_name.iterrows():
        table = row['table_name']
        query = f"""
            select * from {table}
        """
        df_tables = pd.read_sql(query, engine)
        list_data[table] = df_tables
    
    return list_data

data_extracted = extract_data(engine,table_name)

for table in data_extracted:
    print(f"{table} : {type(data_extracted[table])}")
    

products : <class 'pandas.core.frame.DataFrame'>
inventory_tracking : <class 'pandas.core.frame.DataFrame'>
orders : <class 'pandas.core.frame.DataFrame'>
order_details : <class 'pandas.core.frame.DataFrame'>
customers : <class 'pandas.core.frame.DataFrame'>
employees : <class 'pandas.core.frame.DataFrame'>


#### 3. Row & Column Information

In [83]:
def get_shape(engine,table_name):
    query = f"""
        select * from {table_name}
    """
    df_tables = pd.read_sql(query,engine)

    n_rows = df_tables.shape[0]
    n_cols = df_tables.shape[1]
    return n_rows,n_cols


for row,table in table_name.iterrows():
    n_rows, n_cols = get_shape(engine,table['table_name'])
    print(f"{table['table_name']} table has {n_rows} rows and {n_cols} columns ")

products table has 54 rows and 7 columns 
inventory_tracking table has 162 rows and 6 columns 
orders table has 1010 rows and 8 columns 
order_details table has 3022 rows and 7 columns 
customers table has 204 rows and 7 columns 
employees table has 103 rows and 7 columns 


#### 4. Check Data Type

In [76]:
def get_dataType(engine,table_name):
    query = f"""
        select * from {table_name}
    """
    df_tables = pd.read_sql(query,engine)

    list_type = df_tables.dtypes.tolist()
    list_col_name = df_tables.columns.tolist()
    result = {}

    for index,col_name in enumerate(list_col_name):
        if list_type[index] == 'object':
            result[col_name] = "String"
        else:
            result[col_name] = str(list_type[index])
    
    return result

list_table_dataType = {}
for row,table in table_name.iterrows():
    data_type = get_dataType(engine,table['table_name'])
    list_table_dataType[table['table_name']] = data_type

list_table_dataType

{'products': {'product_id': 'int64',
  'product_name': 'String',
  'category': 'String',
  'unit_price': 'String',
  'cost_price': 'String',
  'in_stock': 'bool',
  'created_at': 'datetime64[ns]'},
 'inventory_tracking': {'tracking_id': 'int64',
  'product_id': 'int64',
  'quantity_change': 'int64',
  'change_date': 'datetime64[ns]',
  'reason': 'String',
  'created_at': 'datetime64[ns]'},
 'orders': {'order_id': 'int64',
  'employee_id': 'int64',
  'customer_id': 'float64',
  'order_date': 'datetime64[ns]',
  'total_amount': 'float64',
  'payment_method': 'String',
  'order_status': 'String',
  'created_at': 'datetime64[ns]'},
 'order_details': {'order_detail_id': 'int64',
  'order_id': 'int64',
  'product_id': 'int64',
  'quantity': 'int64',
  'unit_price': 'float64',
  'subtotal': 'float64',
  'created_at': 'datetime64[ns]'},
 'customers': {'customer_id': 'int64',
  'first_name': 'String',
  'last_name': 'String',
  'email': 'String',
  'phone': 'String',
  'loyalty_points': 'int64'

In [65]:
table_name

,table_name
0,products
1,inventory_tracking
2,orders
3,order_details
4,customers
5,employees


#### 5. Check Unique Value

In [84]:
def get_unique_values(engine,table_name):
    query = f"""
        select * from {table_name}
    """
    df_tables = pd.read_sql(query,engine)
    list_col_name = df_tables.columns.tolist()

    result = {}
    for col in list_col_name :
        unique_val = df_tables[col].unique().tolist()
        result[col] = unique_val
    return result


list_check = ['employees','orders','products','inventory_tracking']
for table in list_check:
    res = get_unique_values(engine,table)
    print(table)
    print(res)



employees
{'employee_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103], 'first_name': ['John', 'Gregory', 'Chris', 'Ryan', 'Tyler', 'Sharon', 'Linda', 'Angela', 'David', 'Adam', 'Douglas', 'Amanda', 'Julie', 'Joshua', 'Danielle', 'Michael', 'Nicole', 'Sarah', 'Jessica', 'Brenda', 'Emily', 'Raymond', 'Eddie', 'Mary', 'Christina', 'Kevin', 'Robert', 'Frederick', 'Chase', 'Kristin', 'Valerie', 'Elizabeth', 'Tommy', 'Richard', 'Brian', 'Chad', 'Louis', 'Shannon', 'Alicia', 'Autumn', 'Michelle', 'Megan', 'Kimberly', 'Zachary', 'Eric', 'Amy', 'Dana', 'Rachael', 'Bonnie', 'Stacey', 'Daniel', 'Andrew', 'Ashley', 'Juan', 'Jenna', 'Yolanda

#### 6. Create Report

In [139]:
from datetime import date
import json 
def create_profiling(engine,table_name):
    data_profiling = {
        "person_in_charge":"Vindo",
        "date_profiling":date.today().isoformat()
    }

    result = {}
    for row,table in table_name.iterrows():
        temp = {}
        # get shape 
        n_rows,n_cols = get_shape(engine,table['table_name'])
        temp['shape'] = [n_rows,n_cols]

        #data types 
        data_type = get_dataType(engine,table['table_name'])
        temp['data_type'] = data_type

        # unique values
        unique_val = get_unique_values(engine,table['table_name'])
        temp['unique_values'] = unique_val

        result[table['table_name']] = temp

    data_profiling['result'] = result

    with open("data_profiling.json", "w") as f:
        f.write(json.dumps(data_profiling, indent = 4,default=str))
    return data_profiling

result = create_profiling(engine,table_name)
print(result)


    
    

{'person_in_charge': 'Vindo', 'date_profiling': '2025-10-17', 'result': {'products': {'shape': [54, 7], 'data_type': {'product_id': 'int64', 'product_name': 'String', 'category': 'String', 'unit_price': 'String', 'cost_price': 'String', 'in_stock': 'bool', 'created_at': 'datetime64[ns]'}, 'unique_values': {'product_id': [53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106], 'product_name': ['Pastry situation', 'Pastry sure', 'Pastry statement', 'Salad on', 'Salad really', 'Salad bag', 'Pastry security', 'Smoothie lead', 'Salad people', 'Sandwich role', 'Pastry plan', 'Sandwich international', 'Coffee commercial', 'Sandwich society', 'Smoothie TV', 'Sandwich room', 'Salad ahead', 'Salad east', 'Pastry raise', 'Pastry put', 'Smoothie around', 'Salad land', 'Coffee group', 'Salad truth', 'Sandwich although', 'Tea budget',

## Data Quality

#### 1. Check Missing Values

In [142]:
def check_missing_values(engine,table_name):
    query = f"""
        select * from {table_name}
    """
    df = pd.read_sql(query,engine)
    col_list = df.columns.tolist()
    result = {}

    for col in col_list:
        total_null = df[col].isnull().sum()
        result[col] = int(total_null)
    
    return result


for row,table in table_name.iterrows():
    missing_val = check_missing_values(engine,table['table_name'])
    print(table['table_name'])
    print(missing_val)


products
{'product_id': 0, 'product_name': 0, 'category': 0, 'unit_price': 0, 'cost_price': 0, 'in_stock': 0, 'created_at': 0}
inventory_tracking
{'tracking_id': 0, 'product_id': 0, 'quantity_change': 0, 'change_date': 0, 'reason': 0, 'created_at': 0}
orders
{'order_id': 0, 'employee_id': 0, 'customer_id': 250, 'order_date': 0, 'total_amount': 0, 'payment_method': 0, 'order_status': 0, 'created_at': 0}
order_details
{'order_detail_id': 0, 'order_id': 0, 'product_id': 0, 'quantity': 0, 'unit_price': 0, 'subtotal': 0, 'created_at': 0}
customers
{'customer_id': 0, 'first_name': 0, 'last_name': 0, 'email': 0, 'phone': 4, 'loyalty_points': 0, 'created_at': 0}
employees
{'employee_id': 0, 'first_name': 0, 'last_name': 0, 'hire_date': 0, 'role': 0, 'email': 0, 'created_at': 0}


#### 2. Date Validation

In [134]:
import re

def date_Validation(engine,table_name):
    list_col = {
        "employess":["hire_date"],
        "inventory_tracking":['change_date'],
        "orders":['order_date']
    }
    if table_name not in list_col.keys():
        return {}
       
    patterns = {
        "date": r"^\d{4}-\d{2}-\d{2}$",
        "datetime": r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$"
    }

    query = f"""
        select * from {table_name}
    """
    df = pd.read_sql(query,engine)

    result = {}
    for col in list_col[table_name]:
        if col == "order_date" :
            pattern = patterns["datetime"] 
        else :
            pattern = patterns["date"]
        
        for index,val in enumerate(df[col]):
            if not re.match(pattern, str(val)):
                result[col] = "There's invalid Date Data"
            else :
                result[col] = "Clean"
                
    return result
    

test = date_Validation(engine,'orders')
print(test)

{'order_date': 'Clean'}


#### 3. Numeric Validation

In [136]:
def numeric_validation(engine,table_name):
    list_col = {
        "products":["unit_price","cost_price"],
        "orders":['total_amount'],
        "order_details":['unit_price','quantity','subtotal'],
        "inventory_tracking":['quantity_change'],
        'customers':['loyalty_points']
    }

    if table_name not in list_col.keys():
        return {}
    
    query = f"""
        select * from {table_name}
    """
    df = pd.read_sql(query,engine)
    
    result = {}
    for col in list_col[table_name]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        if df[col].isna().any():
            result[col] = "There's Invalid Numeric Data"
        else:
            result[col] = "Clean"
    
    return result


test = numeric_validation(engine,'products')
print(test)   

{'unit_price': "There's Invalid Numeric Data", 'cost_price': "There's Invalid Numeric Data"}


#### 4.Negative Value


In [137]:
def negative_validation(engine,table_name):
    list_col = {
        "products":["unit_price","cost_price"],
        "orders":['total_amount'],
        "order_details":['unit_price','quantity','subtotal'],
        "inventory_tracking":['quantity_change'],
        'customers':['loyalty_points']
    }

    if table_name not in list_col.keys():
        return {}
    
    query = f"""
        select * from {table_name}
    """
    df = pd.read_sql(query,engine)
    
    result = {}
    for col in list_col[table_name]:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        if (df[col] < 0 ).any():
            result[col] = "There's Negative Value "
        else:
            result[col] = "Clean"
    
    return result


test = negative_validation(engine,'customers')
print(test)   

{'loyalty_points': "There's Negative Value "}


#### 5 Build Report

In [143]:
def create_report (engine,table_name):
    data_report = {
        "person_in_charge":"Vindo",
        "date_profiling":date.today().isoformat()
    }

    result = {}
    for row,table in table_name.iterrows():
        temp = {}

        #missing Value
        missing_val = check_missing_values(engine,table["table_name"])
        temp['missing_value'] = missing_val

        #date validity 
        date_val = date_Validation(engine,table["table_name"])
        temp['date_validity'] = date_val

        #numeric validity
        numeric_val = numeric_validation(engine,table["table_name"])
        temp['numeric_validity'] = numeric_val

        #negative validity
        negative_val = negative_validation(engine,table["table_name"])
        temp['negative_validity'] = negative_val

        result[table["table_name"]] = temp
    
    data_report['result'] = result
    with open("data_report.json", "w") as f:
        f.write(json.dumps(data_report, indent = 4,default=str))
    return data_report

result = create_report(engine,table_name)
print(result)
        


{'person_in_charge': 'Vindo', 'date_profiling': '2025-10-17', 'result': {'products': {'missing_value': {'product_id': 0, 'product_name': 0, 'category': 0, 'unit_price': 0, 'cost_price': 0, 'in_stock': 0, 'created_at': 0}, 'date_validity': {}, 'numeric_validity': {'unit_price': "There's Invalid Numeric Data", 'cost_price': "There's Invalid Numeric Data"}, 'negative_validity': {'unit_price': 'Clean', 'cost_price': 'Clean'}}, 'inventory_tracking': {'missing_value': {'tracking_id': 0, 'product_id': 0, 'quantity_change': 0, 'change_date': 0, 'reason': 0, 'created_at': 0}, 'date_validity': {'change_date': "There's invalid Date Data"}, 'numeric_validity': {'quantity_change': 'Clean'}, 'negative_validity': {'quantity_change': 'Clean'}}, 'orders': {'missing_value': {'order_id': 0, 'employee_id': 0, 'customer_id': 250, 'order_date': 0, 'total_amount': 0, 'payment_method': 0, 'order_status': 0, 'created_at': 0}, 'date_validity': {'order_date': 'Clean'}, 'numeric_validity': {'total_amount': 'Clean

## Recommendation

1. We need to clean numeric columns:
    - products = [unit_price,cost_price]
2. We need to clean date columns :
    - inventory_tracking = [change_date]
3. We need to clean negative values :
    - customers = [loyalty_points]
4. There are null values in several columns
